# Описание задачи
Задача — предсказать следующие 10 MCC-кодов 7033 клиентов, основываясь на их предыдущих тратах.

В качестве тренировочной выборки предоставлены последовательности MCC-кодов  7033 клиентов с таргетом в виде 10 последующих MCC-кодов.

Изначально пытался предсказать 10 следующих MCC кодов, но в результате метрика на Kagle была низкой. Попытавшись подстроиться под метрику на Kagle, понял, что нужно предсказать не 10 следующих MCC кодов, а список из 10 кодов отранжированый по вероятности следующей покупкой.

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")


## Функции

In [ ]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual:
            position_in_actual = actual.index(p)
            # Вводим коэффициент позиции, который учитывает порядок
            position_coefficient = 1.0 / (position_in_actual + 1)
            if p not in predicted[:i]:
                num_hits += 1.0
                score += (num_hits / (i + 1.0)) * position_coefficient

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


## Получение данных

In [ ]:
df_train = pd.read_csv('/kaggle/input/alfabankchallengedata/df_train.csv', sep=';')
df_test = pd.read_csv('/kaggle/input/alfabankchallengedata/df_test.csv', sep=';')

In [ ]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

In [ ]:
# Функция для расширения df_train и добавления ранга
def expand_rows_with_rank(df):
    data = []
    for _, row in df.iterrows():
        rank = 1
        for mcc in row['Target']:
            data.append([row['Id'], mcc, rank])
            rank += 1
    return pd.DataFrame(data, columns=['Id', 'mcc_code', 'rank'])

# Применение функции к df_train
expanded_df = expand_rows_with_rank(df_train)

# Функция для расширения df_test
def expand_rows_without_target(df):
    data = []
    for _, row in df.iterrows():
        for mcc in row['Data']:
            data.append([row['Id'], mcc])
    return pd.DataFrame(data, columns=['Id', 'mcc_code'])

# Применение функции к df_test
expanded_df_test = expand_rows_without_target(df_test)


In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Создаем словарь для быстрого доступа к данным
user_data_dict = expanded_df.groupby('Id')['mcc_code'].apply(list).to_dict()

def mcc_counts_for_user_optimized(user_id, mcc_code):
    user_data = user_data_dict.get(user_id, [])
    return user_data.count(mcc_code)

expanded_df['mcc_count'] = expanded_df.progress_apply(lambda row: mcc_counts_for_user_optimized(row['Id'], row['mcc_code']), axis=1)
expanded_df_test['mcc_count'] = expanded_df_test.progress_apply(lambda row: mcc_counts_for_user_optimized(row['Id'], row['mcc_code']), axis=1)


100%|██████████| 3353026/3353026 [00:45<00:00, 74180.88it/s]


In [ ]:
N = 10

def last_n_counts(mcc_list, n=N):
    if not isinstance(mcc_list, list):
        return {}
    return {mcc: mcc_list[-n:].count(mcc) for mcc in set(mcc_list[-n:])}



# Для expanded_df, предполагая что у вас есть столбец 'Data' в expanded_df с историей MCC кодов для каждого пользователя
expanded_df['last_n_counts'] = df_train['Data'].apply(last_n_counts)
expanded_df_test['last_n_counts'] = df_test['Data'].apply(last_n_counts)

In [ ]:
expanded_df['last_n_counts'] = expanded_df['last_n_counts'].apply(lambda x: x if isinstance(x, dict) else {})
expanded_df_test['last_n_counts'] = expanded_df_test['last_n_counts'].apply(lambda x: x if isinstance(x, dict) else {})

In [ ]:
mcc_cols_train = pd.DataFrame(expanded_df['last_n_counts'].tolist())
mcc_cols_train.columns = ['mcc_' + str(col) for col in mcc_cols_train.columns]
expanded_df = pd.concat([expanded_df.drop(columns=['last_n_counts']), mcc_cols_train], axis=1)
expanded_df.fillna(0, inplace=True)

mcc_cols_test = pd.DataFrame(expanded_df_test['last_n_counts'].tolist())
mcc_cols_test.columns = ['mcc_' + str(col) for col in mcc_cols_test.columns]
expanded_df_test = pd.concat([expanded_df_test.drop(columns=['last_n_counts']), mcc_cols_test], axis=1)
expanded_df_test.fillna(0, inplace=True)


In [ ]:
# Создаем словарь для быстрого доступа к данным
user_data_dict = df_train.set_index('Id')['Data'].to_dict()

def last_occurrence_for_user_optimized(user_id, mcc_code):
    user_data = user_data_dict.get(user_id, [])

    # Проверяем, существует ли mcc_code в user_data
    if mcc_code in user_data:
        return len(user_data) - 1 - user_data[::-1].index(mcc_code)
    else:
        return -1  # или любое другое значение, указывающее на отсутствие mcc_code в user_data

expanded_df['last_occurrence'] = expanded_df.progress_apply(lambda row: last_occurrence_for_user_optimized(row['Id'], row['mcc_code']), axis=1)
expanded_df_test['last_occurrence'] = expanded_df_test.progress_apply(lambda row: last_occurrence_for_user_optimized(row['Id'], row['mcc_code']), axis=1)


100%|██████████| 3353026/3353026 [01:38<00:00, 34149.58it/s]


In [ ]:
def avg_interval_for_user(user_id, mcc_code):
    # Получаем историю MCC кодов для данного пользователя
    user_data = expanded_df[expanded_df['Id'] == user_id]['mcc_code'].tolist()

    # Определите индексы, на которых появляется данный mcc_code
    indices = [i for i, x in enumerate(user_data) if x == mcc_code]

    # Вычислите средний интервал между этими индексами
    intervals = [indices[i+1] - indices[i] for i in range(len(indices)-1)]

    return sum(intervals) / len(intervals) if intervals else -1

expanded_df['avg_interval'] = expanded_df.progress_apply(lambda row: avg_interval_for_user(row['Id'], row['mcc_code']), axis=1)
expanded_df_test['avg_interval'] = expanded_df_test.progress_apply(lambda row: avg_interval_for_user(row['Id'], row['mcc_code']), axis=1)


100%|██████████| 3353026/3353026 [25:34<00:00, 2184.55it/s]


In [ ]:
def trend_last_n_for_user(user_id, mcc_code, n=10):
    mcc_list = user_data_dict.get(user_id, [])

    if len(mcc_list) <= n:
        # Если у пользователя меньше или равно N транзакций, то невозможно определить тренд
        return -1

    last_n = mcc_list[-n:]
    recent_count = last_n.count(mcc_code)
    earlier_count = mcc_list[:-n].count(mcc_code) / (len(mcc_list) - n)

    return 1 if recent_count > earlier_count else 0

expanded_df['trend_last_n'] = expanded_df.progress_apply(lambda row: trend_last_n_for_user(row['Id'], row['mcc_code']), axis=1)
expanded_df_test['trend_last_n'] = expanded_df_test.progress_apply(lambda row: trend_last_n_for_user(row['Id'], row['mcc_code']), axis=1)


100%|██████████| 3353026/3353026 [05:31<00:00, 10102.79it/s]


In [ ]:
X_train = expanded_df.drop(columns=['Id', 'mcc_code', 'rank'])
X_test = expanded_df_test.drop(columns=['Id', 'mcc_code'])
y_train = expanded_df['rank']


In [ ]:
unique_mcc_codes = expanded_df['mcc_code'].unique()
num_unique_mcc = len(unique_mcc_codes)
print(num_unique_mcc)


163


In [ ]:
# Предварительно создаём словарь истинных ранжированных списков MCC-кодов
id_to_true_mcc_list = df_train.set_index('Id')['Target'].to_dict()

group_train = expanded_df.groupby('Id').size().values
train_data = lgb.Dataset(X_train, label=y_train, group=group_train)


def lgb_mapk(preds, train_data):
    # Извлекаем истинные метки
    actuals = train_data.get_label()

    # Преобразование предсказаний в ранжированные списки MCC-кодов
    predicted_scores_per_user = preds.reshape(-1, 163)
    predicted_sorted_indices = np.argsort(predicted_scores_per_user, axis=1)[:, ::-1]
    predicted_mcc_lists = [unique_mcc_codes[indices].tolist() for indices in predicted_sorted_indices]

    # Извлечение истинных ранжированных списков MCC-кодов
    actual_mcc_lists = [id_to_true_mcc_list[id] for id in expanded_df['Id']]

    return 'mapk', mapk(actual_mcc_lists, predicted_mcc_lists), True



In [ ]:
params = {
    'objective': 'lambdarank',
    'metric': 'map@10',
}

model = lgb.train(params, train_data, num_boost_round=100, feval=lgb_mapk)



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000975 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 70330, number of used features: 4


In [ ]:
print("Feature importances:", model.feature_importance())


Feature importances: [ 389 1877  562  172]


In [ ]:
y_test_pred = model.predict(X_test)  # X_test - ваши тестовые данные


In [ ]:
y_test_pred

array([ 0.0242591 ,  0.0242591 ,  0.02298649, ..., -0.07413741,
       -0.00225169,  0.0123122 ])

In [ ]:
expanded_df_test['preds'] = y_test_pred

# Группировка по Id и сортировка внутри каждой группы по предсказаниям
grouped = expanded_df_test.groupby('Id', group_keys=False).apply(lambda x: x.sort_values('preds', ascending=False))

# Выбор топ-N mcc_code для каждого Id
top_n_mcc_per_id = grouped.groupby('Id')['mcc_code'].apply(lambda x: x.head(10).tolist())

print(top_n_mcc_per_id)

Id
0       [5411, 5411, 5411, 5411, 5411, 5411, 6010, 601...
1       [4814, 4814, 4814, 4814, 6010, 6010, 6010, 601...
2       [6011, 6011, 6011, 6011, 6011, 6011, 6011, 601...
3       [5912, 5912, 5912, 5912, 5912, 5912, 5912, 591...
4       [4814, 4814, 6011, 4814, 4814, 4814, 6011, 481...
                              ...                        
7028    [5211, 5211, 5211, 5211, 5211, 5211, 5211, 521...
7029    [5912, 5912, 5912, 5912, 5331, 5331, 5331, 533...
7030    [5200, 5200, 5411, 5411, 5411, 5411, 5912, 541...
7031    [5541, 5541, 5541, 5541, 5541, 5541, 5541, 554...
7032    [4812, 5661, 5661, 5211, 5211, 5814, 8999, 731...
Name: mcc_code, Length: 7033, dtype: object


In [ ]:
final_predictions = top_n_mcc_per_id.reset_index()
final_predictions.columns = ['Id', 'Predicted']
print(final_predictions)


        Id                                          Predicted
0        0  [5411, 5411, 5411, 5411, 5411, 5411, 6010, 601...
1        1  [4814, 4814, 4814, 4814, 6010, 6010, 6010, 601...
2        2  [6011, 6011, 6011, 6011, 6011, 6011, 6011, 601...
3        3  [5912, 5912, 5912, 5912, 5912, 5912, 5912, 591...
4        4  [4814, 4814, 6011, 4814, 4814, 4814, 6011, 481...
...    ...                                                ...
7028  7028  [5211, 5211, 5211, 5211, 5211, 5211, 5211, 521...
7029  7029  [5912, 5912, 5912, 5912, 5331, 5331, 5331, 533...
7030  7030  [5200, 5200, 5411, 5411, 5411, 5411, 5912, 541...
7031  7031  [5541, 5541, 5541, 5541, 5541, 5541, 5541, 554...
7032  7032  [4812, 5661, 5661, 5211, 5211, 5814, 8999, 731...

[7033 rows x 2 columns]


In [ ]:
all_data = df_train['Data'] + df_train['Target'] + df_test['Data']

In [ ]:
total_top_10 = (all_data).explode().value_counts(ascending=False).index[:10]

In [ ]:
sequences_dict = {}

# Обработка df_train
for index, row in df_train.iterrows():
    # Преобразование строки 'Data' и 'Target' в список целых чисел
    data_list = list(map(int, row['Data'].split(','))) if isinstance(row['Data'], str) else row['Data']
    target_list = list(map(int, row['Target'].split(','))) if isinstance(row['Target'], str) else row['Target']
    sequences_dict[row['Id']] = data_list + target_list

# Обработка df_test
for index, row in df_test.iterrows():
    data_list = list(map(int, row['Data'].split(','))) if isinstance(row['Data'], str) else row['Data']
    # Для df_test добавляем только 'Data', так как 'Target' неизвестен
    if row['Id'] in sequences_dict:
        sequences_dict[row['Id']].extend(data_list)
    else:
        sequences_dict[row['Id']] = data_list


In [ ]:
def preprocess_and_postprocess_predictions(final_predictions, total_top_10, sequences_dict):
    # Преобразование строки предсказаний в список уникальных целых чисел
    def process_predicted_string(predicted_str):
        # Удаление скобок и разделение по запятым или пробелам
        predicted_codes = [int(code.strip()) for code in predicted_str.strip('[]').replace(',', ' ').split() if code.strip().isdigit()]
        # Удаление дубликатов, сохраняя порядок
        return list(dict.fromkeys(predicted_codes))

    final_predictions['Predicted'] = final_predictions['Predicted'].apply(process_predicted_string)

    # Функция для дополнения списка MCC кодов до 10 элементов
    def fill_codes(predicted_codes, sequences_dict, total_top_10):
        if len(predicted_codes) >= 10:
            return predicted_codes[:10]

        # Получение списка MCC кодов для данного Id из sequences_dict
        user_sequence = sequences_dict.get(row['Id'], [])
        code_frequency = Counter(user_sequence)

        # Сортировка кодов по частоте встречаемости, исключая уже предсказанные
        most_common_codes = [code for code, freq in code_frequency.most_common() if code not in predicted_codes]

        # Дополнение списка уникальных кодов наиболее часто встречающимися
        for code in most_common_codes:
            if len(predicted_codes) == 10:
                break
            predicted_codes.append(code)
        # Дополнение списка самыми популярными кодами по всему датасету, если нужно
        for code in total_top_10:
            if len(predicted_codes) == 10:
                break
            if code not in predicted_codes:
                predicted_codes.append(code)

        return predicted_codes

    # Применение функции дополнения к каждому набору предсказаний
    for index, row in final_predictions.iterrows():
        final_predictions.at[index, 'Predicted'] = fill_codes(row['Predicted'], sequences_dict, total_top_10)

    return final_predictions


In [ ]:
final = preprocess_and_postprocess_predictions(final_predictions, total_top_10, sequences_dict)

In [ ]:
submission = final[['Id', 'Predicted']]
submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
submission.to_csv('submission_no_repeats_final.csv', index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission_no_repeats_final.csv')


/kaggle/working/submission_no_repeats_final.csv